In [1]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm, trange
import tqdm
from pprint import pprint 
import statsmodels.api as sm
import math
import geemap

from IPython.display import Image

import folium
from folium import plugins
import importlib

import geopandas as gpd
from PIL import Image

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWia6ifsaQpQJBuD6v-u4hmkdbHzY9XuisRY-mnlq3K8RaAv6_Y2g-w

Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
col_sent = ee.ImageCollection("COPERNICUS/S2_SR")

In [39]:
# Isolate Cook County from the Census collection

counties = ee.FeatureCollection('TIGER/2018/Counties')
cookCounty = counties.filter(ee.Filter.eq("GEOID", '17031'))

zipCodes = (
    ee.FeatureCollection('TIGER/2010/ZCTA5').filterBounds(cookCounty)
    .filter("GEOID10 != '46311'").filter("GEOID10 != '46324'")
    .filter("GEOID10 != '46321'").filter("GEOID10 != '46327'")
    .filter("GEOID10 != '46320'")
    
censusTracts = ee.FeatureCollection("TIGER/2010/Tracts_DP1").filterBounds(cookCounty)

zipCodeIDs = zipCodes.aggregate_array("GEOID10").getInfo()
pprint(zipCodeIDs, compact=True)

#censusinChi = censusTracts.filter(ee.Filter.contains('.geo', cookCounty))

zipChiGeom = zipCodes.union(100);

SyntaxError: invalid syntax (<ipython-input-39-31fe89cae389>, line 7)

In [18]:
# Subset images with less than 5% cloud cover, then subset those which intersect the 
# lower left corner of the rectangle 'poly' we've just defined above.
clouds = col_sent.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 
                                      5)).filterBounds(cookCounty)

# Adjust chosen bands to anything you want, and the featurization will flow from it
chosen_bands = ['B2', 'B3', 'B4', 'B8']

# choose only desired bands from the image 
bands = clouds.select(chosen_bands)
s = bands.sort('CLOUD_COVER')
# s contains all sentinel images which satisfied the above two filters 

# Option 2:
scene = s.reduce(ee.Reducer.median())

In [19]:
# Applying the same filters as above, but only to the Near Infared Band (NIR):
nir = clouds.select(['B8']).reduce(ee.Reducer.median())

# Do the same for the red band
red = clouds.select(['B4']).reduce(ee.Reducer.median())

# Calculate NDVI (Normalized Difference Vegetation Index):
numer = nir.add(red)
denom = nir.subtract(red)

ndvi = denom.divide(numer)
ndvi_Chi = ndvi.clip(zipChiGeom.geometry())
#masked = ndvi.updateMask(cookCounty)

ndviVP = {'min': -1, 'max': 1, 'palette': ['000FFF', 'FFFFFF', '00FF00']}

In [20]:
#zipCodes_shp = './ZipShapeFile.shp'
#zipCodes = geemap.shp_to_ee(zipCodes_shp)
#Map = geemap.Map(location=[lby, lbx], zoom_start=11)
fmap = geemap.Map()
fmap.addLayer(ndvi_Chi, ndviVP, 'Scene 1')
fmap.centerObject(cookCounty, 9)
fmap.addLayer(cookCounty)
fmap.addLayer(zipCodes)
#fmap.addLayer(censusinChi)
fmap

EEException: Filter.contains: Unable to use a collection in an algorithm that requires a feature or image. This may happen when trying to use a collection of collections where a collection of features is expected; use flatten, or map a function to convert inner collections to features. Use clipToCollection (instead of clip) to clip an image to a collection.